In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [6]:
!pip install -q sentence-transformers


In [7]:
!pip install -q transformers datasets


In [8]:
print(train_data.columns)
print(test_data.columns)


NameError: name 'train_data' is not defined

In [ ]:
# Now you can import and use the libraries
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers import CrossEncoder
import re

# Load data
train_path = '/kaggle/input/quora-question-pairs/train.csv.zip'
test_path = '/kaggle/input/quora-question-pairs/test.csv.zip'

train_data = pd.read_csv(train_path, low_memory=False)
test_data = pd.read_csv(test_path, low_memory=False)

# Preprocessing function
def preprocess(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

# Apply preprocessing to the data
train_data['question1'] = train_data['question1'].apply(preprocess)
train_data['question2'] = train_data['question2'].apply(preprocess)
test_data['question1'] = test_data['question1'].apply(preprocess)
test_data['question2'] = test_data['question2'].apply(preprocess)

# Convert data into InputExample format for training
train_examples = [InputExample(texts=[row['question1'], row['question2']], label=float(row['is_duplicate'])) for _, row in train_data.iterrows()]

# For the test dataset, we don't have labels, so we skip adding the label
test_examples = [InputExample(texts=[row['question1'], row['question2']]) for _, row in test_data.iterrows()]

# Create DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Bi-Encoder Evaluation Function (For validation)
def evaluate_bi_encoder(model, test_examples):
    predictions = []
    for example in test_examples:
        embeddings1 = model.encode(example.texts[0], convert_to_tensor=True)
        embeddings2 = model.encode(example.texts[1], convert_to_tensor=True)
        cosine_sim = torch.nn.functional.cosine_similarity(embeddings1.unsqueeze(0), embeddings2.unsqueeze(0)).item()
        predictions.append(1 if cosine_sim > 0.5 else 0)
    return predictions

# Train a Bi-Encoder with Cosine Similarity Loss
def train_bi_encoder_with_cosine_loss():
    model_cosine = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    loss_cosine = losses.CosineSimilarityLoss(model=model_cosine)
    model_cosine.fit(train_objectives=[(train_dataloader, loss_cosine)], epochs=3, warmup_steps=100)
    print("Finished training with Cosine Loss.")
    return model_cosine

# Train a Bi-Encoder with Contrastive Loss
def train_bi_encoder_with_contrastive_loss():
    model_contrastive = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    loss_contrastive = losses.ContrastiveLoss(model=model_contrastive)
    model_contrastive.fit(train_objectives=[(train_dataloader, loss_contrastive)], epochs=3, warmup_steps=100)
    print("Finished training with Contrastive Loss.")
    return model_contrastive

# Train a Bi-Encoder with Multiple Negatives Ranking Loss
def train_bi_encoder_with_mnr_loss():
    model_mnr = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    loss_mnr = losses.MultipleNegativesRankingLoss(model=model_mnr)
    model_mnr.fit(train_objectives=[(train_dataloader, loss_mnr)], epochs=3, warmup_steps=100)
    print("Finished training with MNR Loss.")
    return model_mnr

# Train a Cross-Encoder
def train_cross_encoder():
    # Convert test data for inference
    test_pairs = [[row['question1'], row['question2']] for _, row in test_data.iterrows()]

    # Initialize and train Cross-Encoder
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
    cross_encoder.fit(train_dataloader, epochs=3)

    # Evaluation function for Cross-Encoder
    predictions = cross_encoder.predict(test_pairs)
    print(f"Predictions for cross-encoder: {predictions}")

# Execute the training
model_cosine = train_bi_encoder_with_cosine_loss()
model_contrastive = train_bi_encoder_with_contrastive_loss()
model_mnr = train_bi_encoder_with_mnr_loss()
train_cross_encoder()

# Evaluate the models on test data
cosine_predictions = evaluate_bi_encoder(model_cosine, test_examples)
contrastive_predictions = evaluate_bi_encoder(model_contrastive, test_examples)
mnr_predictions = evaluate_bi_encoder(model_mnr, test_examples)

# Print the first 10 predictions for each model
print("Cosine Similarity Predictions:", cosine_predictions[:10])
print("Contrastive Loss Predictions:", contrastive_predictions[:10])
print("MNR Predictions:", mnr_predictions[:10])

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25269 [00:00<?, ?it/s]

Iteration:   0%|          | 0/25269 [00:00<?, ?it/s]